In [ ]:
import time
import time, psutil, os
start = time.time()
m0 = psutil.Process(os.getpid()).memory_info().rss / 1024**2

In [ ]:
import re
import unidecode
import polars as pl
import numpy as np
from rapidfuzz import fuzz
from pathlib import Path

<frozen importlib._bootstrap>:491: RuntimeWarning: The global interpreter lock (GIL) has been enabled to load module '_polars_runtime_32._polars_runtime_32', which has not declared that it can run safely without the GIL. To override this behavior and keep the GIL disabled (at your own risk), run with PYTHON_GIL=0 or -Xgil=0.


# Moradas de Histórico

In [ ]:
import polars as pl

ficheiro = Path.home() / "Downloads" / "06.BNE" / "addressGeoLoc_export_20221006.csv"

df = pl.read_csv(
    ficheiro,
    separator=",",
    quote_char='"',
    skip_rows=1,
    has_header=False,
    new_columns=["MORADA", "CP", "LOCALIDADE", "LATITUDE", "LONGITUDE"],
    decimal_comma=True
)

pl.Config.set_tbl_formatting("UTF8_FULL")  # formato mais limpo
pl.Config.set_tbl_rows(5)                 # nº de linhas a mostrar
pl.Config.set_tbl_width_chars(120) 
pl.Config.set_tbl_cols("10_000")    



polars.config.Config

In [ ]:
df.head()

MORADA,CP,LOCALIDADE,LATITUDE,LONGITUDE
str,str,str,str,str
"""RUA CESARIO VERDE LOTE 3 A DAS…","""2660""","""FRIELAS""","""38.852697456""","""-9.16971532999997"""
"""RUA DO ROXICO NR50""","""3865-110""","""FERMELA""","""40.709808191""","""-8.54878568099997"""
"""AVENIDA 13 DE MAIO N 536""","""3885-227""","""CORTEGACA OVR""","""40.94109138""","""-8.61864986999996"""
"""AV REINALDO SANTOS N 24 3 DTO""","""2675-673""","""ODIVELAS""","""38.7926955990001""","""-9.18881750599996"""
"""RUA DO BARREIRO N 547""","""4405-730""","""VILA NOVA DE GAIA""","""41.113905909""","""-8.63062512299996"""


In [ ]:
df_filtrado = df.filter(
    df["CP"].str.slice(0, 2).is_in(["40"])
)

df_filtrado

MORADA,CP,LOCALIDADE,LATITUDE,LONGITUDE
str,str,str,str,str
"""TRAVESSA DE SA O CARLOS 1 1 FR…","""4050-544""","""PORTO""","""41.15159""","""-8.614601"""
"""RUA DA FIRMEZA 482""","""4000-216""","""PORTO""","""41.1510188030001""","""-8.60600840199993"""
"""RUA DE CAMOES NO93 5OA""","""4000-144""","""PORTO""","""41.152648013""","""-8.61019293599998"""
…,…,…,…,…
"""R NOSSA SENHORA DE FATIMA 322""","""4050-426""","""PORTO""","""41.1594219950001""","""-8.62593197999996"""
"""RUA DE CAMOES 289 3D""","""4000""","""PORTO""","""41.15445""","""-8.610141"""


In [ ]:
import re

# 1️⃣ Número da porta (todas as variantes portuguesas)
regex_numero = re.compile(
    r"""
    (?:\bN[ºO]?[\.]?\s*|     # N, Nº, NO, N.
       NUM(?:ERO)?\.?\s*|    # NUM, NUMERO
       NR\.?\s*|             # NR
       NUMR\.?\s*|           # NUMR
       [^\w]|^               # separador ou início da linha
    )
    (\d{1,4}[A-Z]?|[A-Z]?\d{1,4})  # número da porta (ex: 12, 12A, A12)
    (?!\d)                          # não seguido de outro dígito
    """,
    re.VERBOSE | re.IGNORECASE
)

# 2️⃣ Andar (abrangente)
regex_andar = re.compile(
    r"""
    (?:
        \b(?:R/?C|RES-?DO-?CH(?:AO)?|CAVE|SUBCAVE|LOJA|TRAS|TRASEIRAS|FUNDOS)\b|
        \b\d{1,2}\s*(?:O|º|A|ESQ|DTO|DIR|FRT|TRS|FTE|AND(?:AR)?)\b
    )
    """,
    re.VERBOSE | re.IGNORECASE
)


In [ ]:
# ==========================
# 1. Imports + dicionários
# ==========================
import re
from typing import Optional
import polars as pl

# --- Stopwords PT-PT (lista fixa) ---
STOPWORDS_REGEX = (
    r"\b(a|o|e|é|do|da|de|em|um|uma|para|com|não|os|as|dos|das|no|na|nos|nas|por|ao|à|às|"
    r"pelo|pela|pelos|pelas|num|numa|nuns|numas|dum|duma|duns|dumas|esse|essa|esses|essas|"
    r"este|esta|estes|estas|aquele|aquela|aqueles|aquelas|isto|isso|aquilo|me|te|se|lhe|"
    r"lhes|nos|vos|mim|ti|si|ele|ela|eles|elas|eu|tu|nós|vós|meu|minha|meus|minhas|teu|"
    r"tua|teus|tuas|seu|sua|seus|suas|nosso|nossa|nossos|nossas|vosso|vossa|vossos|vossas|"
    r"que|qual|quais|quem|onde|quando|como|porque|mas|porém|todavia|contudo|entretanto|"
    r"logo|portanto|assim|também|ou|nem|já|ainda|só|mesmo|apenas|até|desde|entre|sob|"
    r"sobre|sem|contra|perante|segundo|durante|após|antes|exceto|salvo|fora|além|"
    r"mediante|conforme|junto|perto)\b"
)

abreviaturas = {
    "cmdt": "comandante", "cmte": "comandante",
    "dr": "doutor", "dra": "doutora", "drs": "doutores",
    "sr": "senhor", "sra": "senhora", "srª": "senhora",
    "eng": "engenheiro", "enga": "engenheira", "engª": "engenheira",
    "prof": "professor", "profa": "professora", "profª": "professora",
    "arq": "arquiteto", "arqa": "arquiteta", "arqª": "arquiteta",
    "cap": "capitão", "maj": "major", "gen": "general",
    "ten": "tenente", "alm": "almirante",
    "sta": "santa", "sto": "santo", "s": "são", "sao": "são",
    "ns": "nossa senhora", "n s": "nossa senhora",
    "nsr": "nosso senhor", "n sr": "nosso senhor",
    "dom": "dom", "d": "dom",
    "visc": "visconde", "cond": "conde", "marq": "marquês", "bar": "barão"
}

mapa_logradouros = {
    "r": "rua", "rua": "rua", "rª": "rua",
    "av": "avenida", "avd": "avenida", "avda": "avenida", "avenida": "avenida",
    "tv": "travessa", "trs": "travessa", "trav": "travessa", "travessa": "travessa",
    "pc": "praça", "pç": "praça", "pr": "praça", "prç": "praça", "praca": "praça",
    "lg": "largo", "al": "alameda", "bq": "beco",
    "esc": "escadas", "estr": "estrada", "cm": "caminho",
    "cç": "calçada", "qt": "quinta", "br": "bairro",
    "rot": "rotunda", "via": "via", "urb": "urbanização",
    "lgd": "lugar", "campo": "campo", "terreiro": "terreiro"
}

# ==========================
# 2. Regex (sem spaCy nem symspellpy)
# ==========================
TITULOS_VALORES = [v.lower() for v in abreviaturas.values()]
TITULOS_REGEX = r"\b(" + "|".join(map(re.escape, sorted(set(TITULOS_VALORES)))) + r")\b"
LOGRADOUROS_REGEX = r"\b(" + "|".join(map(re.escape, sorted(set(mapa_logradouros.values())))) + r")\b"
RUIDO_POS_VIA_REGEX = r"\b(frt|frente|tras|trás|dto|dir|esq|direito|esquerdo|fundos|c/v|cv|bloco|loja|apto|apt|slj|sobreloja|rc|r/c|cave|subcave)\b"

# ==========================
# 3. Auxiliares (correção simples sem dicionário externo)
# ==========================
def corrigir_toponimo(txt: Optional[str]) -> str:
    if not txt or not isinstance(txt, str):
        return ""
    txt = txt.lower().strip()
    # Correções comuns em moradas PT
    correcoes = {
        "camoes": "camões", "camoes": "camões",
        "sao": "são", "sto": "santo", "sta": "santa",
        "ns": "nossa senhora", "nsr": "nosso senhor",
        "alm": "almirante", "gen": "general",
        "cond": "conde", "visc": "visconde", "marq": "marquês",
        "eng": "engenheiro", "arq": "arquiteto",
        "prof": "professor", "dr": "doutor"
    }
    palavras = txt.split()
    corrigidas = [correcoes.get(p, p) for p in palavras]
    # Remove repetições
    dedup = []
    for p in corrigidas:
        if not dedup or p != dedup[-1]:
            dedup.append(p)
    return " ".join(dedup)

def limpar_repeticoes(txt: str) -> str:
    if not txt:
        return ""
    parts, last = [], None
    for p in txt.split():
        if p != last:
            parts.append(p)
            last = p
    return " ".join(parts)

# ==========================
# 4. Função principal
# ==========================
def extrair_partes(df: pl.DataFrame) -> pl.DataFrame:
    # ---------- Normalização ----------
    df = df.with_columns(
        pl.col("MORADA")
          .str.to_lowercase()
          .str.replace_all(r"[\.]", "")
          .str.replace_all(r"\s+", " ")
          .str.strip_chars()
          .alias("morada_norm")
    )

    # ---------- Tipo de logradouro ----------
    tipo_pat = r"^(r\.?|rua|av\.?|avenida|tv\.?|travessa|pc\.?|pç\.?|praça|estr\.?|estrada|al\.?|alameda|lg\.?|largo|cç\.?|calçada|rotunda)"
    art_tipo = (
        pl.col("morada_norm")
          .str.extract(tipo_pat, 1)
          .replace(mapa_logradouros, default=None)
          .fill_null("")
    )

    # ---------- Nome da via ----------
    nome_via = (
        pl.col("morada_norm")
          .str.replace(tipo_pat + r"\s+", "", literal=False)
          .str.replace_all(r"\b(?:sa\s*o|s\s*o)\b", "são")
          .str.extract(r"^([^\d]+?)(?:\d|$)", 1)
          .str.strip_chars()
          .fill_null("")
    )

    # ---------- Expandir abreviaturas + dedup ----------
    nome_expandido = nome_via
    for abrev, completo in abreviaturas.items():
        nome_expandido = nome_expandido.str.replace_all(rf"\b{abrev}\b", completo)
    nome_expandido = nome_expandido.map_elements(limpar_repeticoes, return_dtype=pl.Utf8)

    # ---------- Título + designação ----------
    art_titulo = nome_expandido.str.extract(TITULOS_REGEX, 1)
    art_desig_base = (
        nome_expandido
          .str.replace(TITULOS_REGEX, "", literal=False)
          .str.replace(LOGRADOUROS_REGEX, "", literal=False)
          .str.replace(STOPWORDS_REGEX, "", literal=False)
          .str.replace_all(RUIDO_POS_VIA_REGEX, "", literal=False)
          .str.replace_all(r"\b[a-z]\b", "", literal=False)
          .str.replace_all(r"\b(n|nr|no|nº|num)\b", "", literal=False)
          .str.replace_all(r"\d+", "", literal=False)
          .str.replace_all(r"\s+", " ")
          .str.strip_chars()
          .fill_null("")
          .map_elements(limpar_repeticoes, return_dtype=pl.Utf8)
    )

    # ---------- Correção final + dedup ----------
    df = df.with_columns(
        art_desig_base
          .map_elements(corrigir_toponimo, return_dtype=pl.Utf8)
          .map_elements(limpar_repeticoes, return_dtype=pl.Utf8)
          .alias("ART_DESIG")
    )

    # ---------- Números ----------
    numeros = pl.col("morada_norm").str.extract_all(r"\d{1,4}")
    porta_num = numeros.list.first()
    andar     = numeros.list.slice(1, 1).list.first()

    # ---------- Código postal ----------
    df = df.with_columns([
        pl.col("CP").str.extract(r"^(\d{4})", 1).alias("CP4"),
        pl.col("CP").str.extract(r"-(\d{3})", 1).alias("CP3"),
        pl.lit("").alias("CPALF")
    ])

    # ---------- Campos finais ----------
    df = df.with_columns([
        pl.lit(None).cast(pl.Utf8).alias("ART_COD"),
        art_tipo.alias("ART_TIPO"),
        art_titulo.alias("ART_TITULO"),
        porta_num.alias("PORTA_NUM"),
        andar.alias("ANDAR"),
        pl.col("LONGITUDE").alias("LONG_PORTA"),
        pl.col("LATITUDE").alias("LAT_PORTA")
    ])

    # ---------- Seleção final ----------
    return df.select([
        "MORADA", "CP", "LOCALIDADE",
        "ART_COD", "ART_TIPO", "ART_TITULO", "ART_DESIG",
        "PORTA_NUM", "ANDAR",
        "CP4", "CP3", "CPALF",
        "LONG_PORTA", "LAT_PORTA"
    ])

In [ ]:
df_partes = extrair_partes(df_filtrado)
df_partes


/var/folders/8l/z5bjd49s42b04x3cy5cc16mh0000gn/T/ipykernel_14814/3361799727.py:112: DeprecationWarning: the `default` parameter for `replace` is deprecated. Use `replace_strict` instead to set a default while replacing values.
(Deprecated in version 1.0.0)
  .replace(mapa_logradouros, default=None)


MORADA,CP,LOCALIDADE,ART_COD,ART_TIPO,ART_TITULO,ART_DESIG,PORTA_NUM,ANDAR,CP4,CP3,CPALF,LONG_PORTA,LAT_PORTA
str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""TRAVESSA DE SA O CARLOS 1 1 FR…","""4050-544""","""PORTO""",null,"""travessa""","""são""","""carlos""","""1""","""1""","""4050""","""544""","""""","""-8.614601""","""41.15159"""
"""RUA DA FIRMEZA 482""","""4000-216""","""PORTO""",null,"""rua""",null,"""firmeza""","""482""",null,"""4000""","""216""","""""","""-8.60600840199993""","""41.1510188030001"""
"""RUA DE CAMOES NO93 5OA""","""4000-144""","""PORTO""",null,"""rua""",null,"""camões""","""93""","""5""","""4000""","""144""","""""","""-8.61019293599998""","""41.152648013"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""R NOSSA SENHORA DE FATIMA 322""","""4050-426""","""PORTO""",null,"""rua""","""nossa senhora""","""fatima""","""322""",null,"""4050""","""426""","""""","""-8.62593197999996""","""41.1594219950001"""
"""RUA DE CAMOES 289 3D""","""4000""","""PORTO""",null,"""rua""",null,"""camões""","""289""","""3""","""4000""",null,"""""","""-8.610141""","""41.15445"""


In [ ]:
import numpy as np

def haversine(lat1, lon1, lat2, lon2):
    # converter para radianos
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    # fórmula haversine
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2
    c = 2 * np.arcsin(np.sqrt(a))
    R = 6371000  # raio da Terra em metros
    return R * c


# Moradas Base Nacional de Endereços (BNE) 

In [ ]:

ficheiro = Path.home() / "Downloads" / "06.BNE" /  "df_Principal_f.csv"

df_BNE = pl.read_csv(
    ficheiro,
    separator=",",
    quote_char='"',
    skip_rows=1,
    has_header=False,
    new_columns=["ART_COD",	"ART_TIPO","ART_TITULO","ART_DESIG","PORTA_NUM","CP4","CP3","CPALF","LOCALIDADE","LONG_PORTA","LAT_PORTA"],
    decimal_comma=True
)

In [ ]:
df_BNE


ART_COD,ART_TIPO,ART_TITULO,ART_DESIG,PORTA_NUM,CP4,CP3,CPALF,LOCALIDADE,LONG_PORTA,LAT_PORTA
i64,str,str,str,i64,i64,i64,str,str,str,str
61213,"""Rua""",null,"""Abraos""",49,4000,12,"""PORTO""","""Porto""","""-8.59""","""41.15"""
61213,"""Rua""",null,"""Abraos""",32,4000,12,"""PORTO""","""Porto""","""-8.59""","""41.15"""
61213,"""Rua""",null,"""Abraos""",10,4000,12,"""PORTO""","""Porto""","""-8.59""","""41.15"""
…,…,…,…,…,…,…,…,…,…,…
2147140000,"""Beco""",null,"""Passos Manuel""",null,4000,7,"""PORTO""","""Porto""","""-8.6""","""41.14"""
2147140000,"""Beco""",null,"""Passos Manuel""",null,4000,7,"""PORTO""","""Porto""","""-8.6""","""41.14"""


In [ ]:
df_BNE_filtrado = df_BNE.filter(pl.col("ART_COD") == 3991213)

df_BNE_filtrado

ART_COD,ART_TIPO,ART_TITULO,ART_DESIG,PORTA_NUM,CP4,CP3,CPALF,LOCALIDADE,LONG_PORTA,LAT_PORTA
i64,str,str,str,i64,i64,i64,str,str,str,str
3991213,"""Rua""",null,"""Cames""",19,4000,144,"""PORTO""","""Porto""","""-8.61""","""41.15"""
3991213,"""Rua""",null,"""Cames""",21,4000,144,"""PORTO""","""Porto""","""-8.61""","""41.15"""
3991213,"""Rua""",null,"""Cames""",29,4000,144,"""PORTO""","""Porto""","""-8.61""","""41.15"""
…,…,…,…,…,…,…,…,…,…,…
3991213,"""Rua""",null,"""Cames""",null,4000,143,"""PORTO""","""Porto""","""-8.6""","""41.15"""
3991213,"""Rua""",null,"""Cames""",null,4000,143,"""PORTO""","""Porto""","""-8.6""","""41.15"""


In [ ]:
# ==========================
# 1. Normalização BNE
# ==========================
df_BNE_norm = df_BNE.with_columns([
    pl.col("ART_TIPO").str.to_lowercase().alias("ART_TIPO"),       # minúsculas em tipo
    pl.col("ART_TITULO").str.to_lowercase().alias("ART_TITULO"),   # minúsculas em título
    pl.col("ART_DESIG").str.to_lowercase().alias("ART_DESIG"),     # minúsculas em designação
    pl.col("LOCALIDADE").str.to_lowercase().alias("LOCALIDADE")    # minúsculas em localidade
])

In [ ]:
df_BNE_norm

ART_COD,ART_TIPO,ART_TITULO,ART_DESIG,PORTA_NUM,CP4,CP3,CPALF,LOCALIDADE,LONG_PORTA,LAT_PORTA
i64,str,str,str,i64,i64,i64,str,str,str,str
61213,"""rua""",null,"""abraos""",49,4000,12,"""PORTO""","""porto""","""-8.59""","""41.15"""
61213,"""rua""",null,"""abraos""",32,4000,12,"""PORTO""","""porto""","""-8.59""","""41.15"""
61213,"""rua""",null,"""abraos""",10,4000,12,"""PORTO""","""porto""","""-8.59""","""41.15"""
…,…,…,…,…,…,…,…,…,…,…
2147140000,"""beco""",null,"""passos manuel""",null,4000,7,"""PORTO""","""porto""","""-8.6""","""41.14"""
2147140000,"""beco""",null,"""passos manuel""",null,4000,7,"""PORTO""","""porto""","""-8.6""","""41.14"""


In [ ]:

# ==========================
# 2. Constantes
# ==========================
R = 6371000.0
p = np.pi / 180.0
limite_deg = 0.005  # ~500m

# ==========================
# 3. Função para limpar coordenadas
# ==========================
def clean_coord(col):
    # substitui vírgulas → ponto e converte para float
    return (
        pl.col(col)
          .cast(pl.Utf8)
          .str.replace(",", ".")
          .replace(["None","nan","NaN",""], None)
          .cast(pl.Float64)
    )

# ==========================
# 4. Preparar bases (lazy → eager)
# ==========================
df_portas = (
    df_partes
    .with_columns([
        pl.col("CP").str.slice(0, 2).alias("CP2"),
        clean_coord("LAT_PORTA").alias("LAT_PORTA"),
        clean_coord("LONG_PORTA").alias("LONG_PORTA")
    ])
    .drop_nulls(["LAT_PORTA", "LONG_PORTA", "CP2"])
)

df_BNE_norm = (
    df_BNE
    .with_columns([
        pl.col("CP4").cast(pl.Utf8).str.slice(0, 2).alias("CP2"),
        clean_coord("LAT_PORTA").alias("LAT_PORTA"),
        clean_coord("LONG_PORTA").alias("LONG_PORTA")
    ])
    .filter(pl.col("CP2") == "40")  # <--- filtro coerente
    .drop_nulls(["LAT_PORTA", "LONG_PORTA", "CP2"])
)



In [ ]:
df_BNE_norm

ART_COD,ART_TIPO,ART_TITULO,ART_DESIG,PORTA_NUM,CP4,CP3,CPALF,LOCALIDADE,LONG_PORTA,LAT_PORTA,CP2
i64,str,str,str,i64,i64,i64,str,str,f64,f64,str
61213,"""Rua""",null,"""Abraos""",49,4000,12,"""PORTO""","""Porto""",-8.59,41.15,"""40"""
61213,"""Rua""",null,"""Abraos""",32,4000,12,"""PORTO""","""Porto""",-8.59,41.15,"""40"""
61213,"""Rua""",null,"""Abraos""",10,4000,12,"""PORTO""","""Porto""",-8.59,41.15,"""40"""
…,…,…,…,…,…,…,…,…,…,…,…
2147140000,"""Beco""",null,"""Passos Manuel""",null,4000,7,"""PORTO""","""Porto""",-8.6,41.14,"""40"""
2147140000,"""Beco""",null,"""Passos Manuel""",null,4000,7,"""PORTO""","""Porto""",-8.6,41.14,"""40"""


In [ ]:
# ==========================
# 5. Porta BNE – k=10 vizinhos mais próximos (<= 500 m)
# ==========================
import numpy as np
import polars as pl
from sklearn.neighbors import BallTree

# -------------------------------------------------
# Configurações
# -------------------------------------------------
R = 6371000.0
RAIO_M = 500.0
K_NEIGHBORS = 500   

# -------------------------------------------------
# 5.1 Coordenadas
# -------------------------------------------------
coords_portas = np.radians(df_portas.select(["LAT_PORTA", "LONG_PORTA"]).to_numpy())
coords_bne = np.radians(df_BNE_norm.select(["LAT_PORTA", "LONG_PORTA"]).to_numpy())

# -------------------------------------------------
# 5.2 Busca k vizinhos
# -------------------------------------------------
tree = BallTree(coords_bne, metric="haversine")
distancias, indices = tree.query(coords_portas, k=K_NEIGHBORS)

# Flatten com reshape
dist_m = (distancias * R).flatten()
idx_bne = indices.flatten().astype(np.uint32)

# Índice da porta (repetido K vezes)
idx_porta = np.repeat(np.arange(len(df_portas), dtype=np.uint32), K_NEIGHBORS)

# -------------------------------------------------
# 5.3 Criar DataFrame + filtro
# -------------------------------------------------
pairs = (
    pl.DataFrame({
        "idx_porta": idx_porta,
        "idx_bne":   idx_bne,
        "dist_m":    dist_m
    })
    .filter(pl.col("dist_m") <= RAIO_M)
    .join(df_portas.with_row_count("idx_porta"), on="idx_porta", how="left")
    .join(df_BNE_norm.with_row_count("idx_bne"),    on="idx_bne",    how="left", suffix="_BNE")
    .drop(["idx_porta", "idx_bne"])
)

# -------------------------------------------------
# 6. Resultado
# -------------------------------------------------
print(f"Total de matches (k={K_NEIGHBORS}): {len(pairs)}")
pairs

/var/folders/8l/z5bjd49s42b04x3cy5cc16mh0000gn/T/ipykernel_14814/2245924403.py:44: DeprecationWarning: `DataFrame.with_row_count` is deprecated; use `with_row_index` instead. Note that the default column name has changed from 'row_nr' to 'index'.
  .join(df_portas.with_row_count("idx_porta"), on="idx_porta", how="left")
/var/folders/8l/z5bjd49s42b04x3cy5cc16mh0000gn/T/ipykernel_14814/2245924403.py:45: DeprecationWarning: `DataFrame.with_row_count` is deprecated; use `with_row_index` instead. Note that the default column name has changed from 'row_nr' to 'index'.
  .join(df_BNE_norm.with_row_count("idx_bne"),    on="idx_bne",    how="left", suffix="_BNE")


Total de matches (k=500): 10845312


dist_m,MORADA,CP,LOCALIDADE,ART_COD,ART_TIPO,ART_TITULO,ART_DESIG,PORTA_NUM,ANDAR,CP4,CP3,CPALF,LONG_PORTA,LAT_PORTA,CP2,ART_COD_BNE,ART_TIPO_BNE,ART_TITULO_BNE,ART_DESIG_BNE,PORTA_NUM_BNE,CP4_BNE,CP3_BNE,CPALF_BNE,LOCALIDADE_BNE,LONG_PORTA_BNE,LAT_PORTA_BNE,CP2_BNE
f64,str,str,str,str,str,str,str,str,str,str,str,str,f64,f64,str,i64,str,str,str,i64,i64,i64,str,str,f64,f64,str
423.864204,"""TRAVESSA DE SA O CARLOS 1 1 FR…","""4050-544""","""PORTO""",null,"""travessa""","""são""","""carlos""","""1""","""1""","""4050""","""544""","""""",-8.614601,41.15159,"""40""",3991213,"""Rua""",null,"""Cames""",295,4000,143,"""PORTO""","""Porto""",-8.61,41.15,"""40"""
423.864204,"""TRAVESSA DE SA O CARLOS 1 1 FR…","""4050-544""","""PORTO""",null,"""travessa""","""são""","""carlos""","""1""","""1""","""4050""","""544""","""""",-8.614601,41.15159,"""40""",3991213,"""Rua""",null,"""Cames""",801,4000,140,"""PORTO""","""Porto""",-8.61,41.15,"""40"""
423.864204,"""TRAVESSA DE SA O CARLOS 1 1 FR…","""4050-544""","""PORTO""",null,"""travessa""","""são""","""carlos""","""1""","""1""","""4050""","""544""","""""",-8.614601,41.15159,"""40""",9831213,"""Rua""",null,"""Gonalo Cristvo""",301,4000,267,"""PORTO""","""Porto""",-8.61,41.15,"""40"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
494.958229,"""RUA DE CAMOES 289 3D""","""4000""","""PORTO""",null,"""rua""",null,"""camões""","""289""","""3""","""4000""",null,"""""",-8.610141,41.15445,"""40""",3991213,"""Rua""",null,"""Cames""",235,4000,143,"""PORTO""","""Porto""",-8.61,41.15,"""40"""
494.958229,"""RUA DE CAMOES 289 3D""","""4000""","""PORTO""",null,"""rua""",null,"""camões""","""289""","""3""","""4000""",null,"""""",-8.610141,41.15445,"""40""",3991213,"""Rua""",null,"""Cames""",247,4000,143,"""PORTO""","""Porto""",-8.61,41.15,"""40"""


# SAE

In [ ]:
# ==========================
# 1. Importar dependências
# ==========================
import polars as pl
from rapidfuzz import fuzz
import math

# ==========================
# 2. Função: distância geográfica (Haversine)
# ==========================
def haversine(lat1, lon1, lat2, lon2):
    R = 6371000  # raio da Terra (m)
    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)
    a = math.sin(dphi/2)**2 + math.cos(phi1)*math.cos(phi2)*math.sin(dlambda/2)**2
    return R * 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

# ==========================
# 3. Função: cálculo do score fuzzy + distância
# ==========================
def calcular_score(row):
    # Comparação texto → normaliza None/NaN para ""
    f = fuzz.token_sort_ratio
    s_cod   = f(str(row["ART_COD"] or ""), str(row["ART_COD_BNE"] or "")) / 100
    s_tipo  = f(str(row["ART_TIPO"] or ""), str(row["ART_TIPO_BNE"] or "")) / 100
    s_tit   = f(str(row["ART_TITULO"] or ""), str(row["ART_TITULO_BNE"] or "")) / 100
    s_desig = f(str(row["ART_DESIG"] or ""), str(row["ART_DESIG_BNE"] or "")) / 100
    s_porta = f(str(row["PORTA_NUM"] or ""), str(row["PORTA_NUM_BNE"] or "")) / 100
    s_andar = f(str(row["ANDAR"] or ""), str(row["ANDAR_BNE"] or "")) / 100 if "ANDAR_BNE" in row else 1
    s_cp4   = f(str(row["CP4"] or ""), str(row["CP4_BNE"] or "")) / 100
    s_cp3   = f(str(row["CP3"] or ""), str(row["CP3_BNE"] or "")) / 100
    s_cpalf = f(str(row["CPALF"] or ""), str(row["CPALF_BNE"] or "")) / 100

    # Distância entre coordenadas
    dist = haversine(
        row["LAT_PORTA"], row["LONG_PORTA"],
        row["LAT_PORTA_BNE"], row["LONG_PORTA_BNE"]
    )

    # Normalização da distância (máx. 500 m)
    s_dist = max(0.0, 1.0 - dist / 500)

    # Score total (peso maior no fuzzy textual)
    score = (
        0.25 * s_desig +
        0.10 * s_tipo +
        0.05 * s_tit +
        0.10 * s_porta +
        0.05 * s_andar +
        0.10 * s_cp4 +
        0.05 * s_cp3 +
        0.05 * s_cpalf +
        0.25 * s_dist
    )
    return {"score": score, "dist_m": dist}

# ==========================
# 4. Aplicar funções
# ==========================
pairs_scored = pairs.with_columns(
    pl.struct(pairs.columns).map_elements(
        lambda row: calcular_score(row)["score"], return_dtype=pl.Float64
    ).alias("score"),
    pl.struct(pairs.columns).map_elements(
        lambda row: calcular_score(row)["dist_m"], return_dtype=pl.Float64
    ).alias("dist_calc")
)

# ==========================
# 5. Escolher melhor match por morada
# ==========================
melhores_matches = (
    pairs_scored
    .sort(["MORADA", "score", "dist_calc"], descending=[False, True, False])
    .group_by("MORADA")
    .agg(pl.all().first())  # 1ª linha: melhor score / menor distância
    .sort("score", descending=True)
)

# ==========================
# 6. Resultado final
# ==========================
melhores_matches


MORADA,dist_m,CP,LOCALIDADE,ART_COD,ART_TIPO,ART_TITULO,ART_DESIG,PORTA_NUM,ANDAR,CP4,CP3,CPALF,LONG_PORTA,LAT_PORTA,CP2,ART_COD_BNE,ART_TIPO_BNE,ART_TITULO_BNE,ART_DESIG_BNE,PORTA_NUM_BNE,CP4_BNE,CP3_BNE,CPALF_BNE,LOCALIDADE_BNE,LONG_PORTA_BNE,LAT_PORTA_BNE,CP2_BNE,score,dist_calc
str,f64,str,str,str,str,str,str,str,str,str,str,str,f64,f64,str,i64,str,str,str,i64,i64,i64,str,str,f64,f64,str,f64,f64
"""RUA DE CAMOES 895 1C DIREITO""",2.132322,"""4000-149""","""PORTO""",null,"""rua""",null,"""camões""","""895""","""1""","""4000""","""149""","""""",-8.61002,41.160011,"""40""",3991213,"""Rua""",null,"""Cames""",895,4000,149,"""PORTO""","""Porto""",-8.61,41.16,"""40""",0.847419,2.132322
"""RUA DE CAMOES 895 2 DIREITO""",2.132322,"""4000-142""","""PORTO""",null,"""rua""",null,"""camões""","""895""","""2""","""4000""","""142""","""""",-8.61002,41.160011,"""40""",3991213,"""Rua""",null,"""Cames""",895,4000,142,"""PORTO""","""Porto""",-8.61,41.16,"""40""",0.847419,2.132322
"""RUA DE CAMOES 895 2 DIREITO FR…",2.132322,"""4000-142""","""PORTO""",null,"""rua""",null,"""camões""","""895""","""2""","""4000""","""142""","""""",-8.61002,41.160011,"""40""",3991213,"""Rua""",null,"""Cames""",895,4000,142,"""PORTO""","""Porto""",-8.61,41.16,"""40""",0.847419,2.132322
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""665 RUA DA ALEGRIA 665""",407.567262,"""4000""","""PORTO""",null,"""""",null,"""""","""665""","""665""","""4000""",null,"""""",-8.602251,41.15675,"""40""",731213,"""Rua""",null,"""Alegria""",961,4000,39,"""PORTO""","""Porto""",-8.6,41.16,"""40""",0.27955,407.567262
"""DAMIAO DE GOIS 446 1 LEFT""",492.423347,"""4050""","""PORTO""",null,"""""",null,"""damiao gois""","""446""","""1""","""4050""",null,"""""",-8.6149,41.16245,"""40""",3991213,"""Rua""",null,"""Cames""",907,4000,149,"""PORTO""","""Porto""",-8.61,41.16,"""40""",0.272538,492.423347


In [ ]:
melhores_80 = melhores_matches.filter(pl.col("score") >= 0.7)

melhores_80

MORADA,dist_m,CP,LOCALIDADE,ART_COD,ART_TIPO,ART_TITULO,ART_DESIG,PORTA_NUM,ANDAR,CP4,CP3,CPALF,LONG_PORTA,LAT_PORTA,CP2,ART_COD_BNE,ART_TIPO_BNE,ART_TITULO_BNE,ART_DESIG_BNE,PORTA_NUM_BNE,CP4_BNE,CP3_BNE,CPALF_BNE,LOCALIDADE_BNE,LONG_PORTA_BNE,LAT_PORTA_BNE,CP2_BNE,score,dist_calc
str,f64,str,str,str,str,str,str,str,str,str,str,str,f64,f64,str,i64,str,str,str,i64,i64,i64,str,str,f64,f64,str,f64,f64
"""RUA DE CAMOES 895 1C DIREITO""",2.132322,"""4000-149""","""PORTO""",null,"""rua""",null,"""camões""","""895""","""1""","""4000""","""149""","""""",-8.61002,41.160011,"""40""",3991213,"""Rua""",null,"""Cames""",895,4000,149,"""PORTO""","""Porto""",-8.61,41.16,"""40""",0.847419,2.132322
"""RUA DE CAMOES 895 2 DIREITO""",2.132322,"""4000-142""","""PORTO""",null,"""rua""",null,"""camões""","""895""","""2""","""4000""","""142""","""""",-8.61002,41.160011,"""40""",3991213,"""Rua""",null,"""Cames""",895,4000,142,"""PORTO""","""Porto""",-8.61,41.16,"""40""",0.847419,2.132322
"""RUA DE CAMOES 895 2 DIREITO FR…",2.132322,"""4000-142""","""PORTO""",null,"""rua""",null,"""camões""","""895""","""2""","""4000""","""142""","""""",-8.61002,41.160011,"""40""",3991213,"""Rua""",null,"""Cames""",895,4000,142,"""PORTO""","""Porto""",-8.61,41.16,"""40""",0.847419,2.132322
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""RUA DA ALEGRIA 828 3 ESQUERDO""",193.557562,"""4000""","""PORTO""",null,"""rua""",null,"""alegria""","""828""","""3""","""4000""",null,"""""",-8.601361,41.158593,"""40""",731213,"""Rua""",null,"""Alegria""",988,4000,38,"""PORTO""","""Porto""",-8.6,41.16,"""40""",0.70084,193.557562
"""PC ALEGRIA N 97 PORTO 4000 030…",340.761094,"""4000-030""","""PORTO""",null,"""praça""",null,"""alegria""","""97""","""4000""","""4000""","""030""","""""",-8.600259,41.143058,"""40""",721213,"""Praa""",null,"""Alegria""",97,4000,30,"""PORTO""","""Porto""",-8.6,41.14,"""40""",0.700572,340.761094


In [ ]:
m1 = psutil.Process(os.getpid()).memory_info().rss / 1024**2
print("Tempo Polars:", round(time.time() - start, 2), "segundos")
print("Memória Polars:", round(m1 - m0, 2), "MB")

Tempo Polars: 197.7 segundos
Memória Polars: 2551.7 MB
